In [3]:
from __future__ import division
import requests
import time as t
from mpl_toolkits.basemap import Basemap
from shapely.geometry import LineString
from matplotlib import pyplot as plt
import numpy as np
import os
import urllib
import json
%matplotlib inline

# Env vars and functions

In [4]:
apiKey = os.environ.get('MAPZEN_API')

def decode(encoded):
    inv = 1.0 / 1e6
    decoded = []
    previous = [0,0]
    i = 0
    while i < len(encoded):
        ll = [0,0]
        for j in [0, 1]:
            shift = 0
            byte = 0x20
            while byte >= 0x20:
                byte = ord(encoded[i]) - 63
                i += 1
                ll[j] |= (byte & 0x1f) << shift
                shift += 5
            ll[j] = previous[j] + (~(ll[j] >> 1) if ll[j] & 1 else (ll[j] >> 1))
            previous[j] = ll[j]
        decoded.append([float('%.6f' % (ll[1] * inv)), float('%.6f' % (ll[0] * inv))])
    return decoded


def synthesize_gps(edges, shape, distribution="normal", stddev=0, uuid='999999'):
    
    jsonDict = {"uuid":"100343", "trace":[]}
    coords = decode(shape)
    sttm = t.time() - 100000
    
    for i, edge in enumerate(edges):
        dist = edge['length']
        speed = edge['speed']
        beginShapeIndex = edge['begin_shape_index']
        endShapeIndex = edge['end_shape_index']
        lon, lat = coords[endShapeIndex]
        if stddev > 0:
            avgLat = np.mean(np.array(coords)[:, 1])
            stddevLon = stddev / 111.111     # approx. 111.111 km per deg lon unless very close to the poles
            stddevLat = stddev / (111.111 * np.cos(avgLat))     # approx 111.111 km * cos(lat) per deg lat
            lon += np.random.normal(scale=stddevLon)
            lat += np.random.normal(scale=stddevLat)
        dur = dist / speed * 3600.0
        time = sttm + dur
        time = int(round(time))
        if i == 0:
            st_lon, st_lat = coords[beginShapeIndex]
            jsonDict["trace"].append({"lat":st_lat,"lon":st_lon,"time":time})
        jsonDict["trace"].append({"lat": lat,"lon":lon,"time":time})
        sttm = time

    return jsonDict

# Open Traffic Reporter Validation Service 

### 1. Get New Route

In [5]:
# 349 38th St Oakland to 111 New Montgomery SF
stLat = 37.8261
stLon = -122.25872
endLat = 37.790168
endLon = -122.402264

jsonDict = {"locations":[
        {"lat":stLat,"lon":stLon,"type":"break"},
        {"lat":endLat,"lon":endLon,"type":"break"}],
            "costing":"auto",
            "id":"my_work_route"}

In [6]:
baseUrl = 'http://valhalla:8002/route'
route = requests.post(baseUrl, json=jsonDict)

In [7]:
print route.status_code
shape = route.json()['trip']['legs'][0]['shape']

200


In [36]:
route.json()['trip']['legs'][0]['maneuvers'][0]

{u'begin_shape_index': 0,
 u'end_shape_index': 1,
 u'instruction': u'Drive southwest on Manila Avenue.',
 u'length': 0.165,
 u'street_names': [u'Manila Avenue'],
 u'time': 16,
 u'travel_mode': u'drive',
 u'travel_type': u'car',
 u'type': 3,
 u'verbal_pre_transition_instruction': u'Drive southwest on Manila Avenue for 200 meters.'}

### 2. Pass Route Shape to Map Matching API 

In [8]:
payload = {
    "encoded_polyline": shape,
    "costing": "pedestrian",
    "directions_options": {
        "units": "kilometers"
    },
    "shape_match": "map_snap",
    "trace_options": {
        "turn_penalty_factor": 500
    }
}

In [9]:
baseUrl = 'http://valhalla:8002/trace_attributes?'

In [10]:
matched = requests.post(baseUrl, json=payload)

In [11]:
print 'Matched {0} points. Showing the first below: \n{1}'.format(
    len(matched.json()['matched_points']), matched.json()['matched_points'][0])

Matched 313 points. Showing the first below: 
{u'lon': -122.258926, u'distance_from_trace_point': 0.0, u'distance_along_edge': 0.119, u'type': u'matched', u'lat': 37.82618, u'edge_index': 0}


In [12]:
tr = requests.post('http://valhalla:8002/trace_routes?', json=payload)

In [15]:
edges = matched.json()['edges']
trafficSegments = [x['traffic_segments'] for x in edges if 'traffic_segments' in x.keys()]
trafficSegmentIDs = [x[0]['segment_id'] for x in trafficSegments]
unqTrafficSegmentIDs = np.unique(trafficSegmentIDs)

In [63]:
print 'Got {0} edges, {1} of which have traffic segments, of which {2} are unique.'.format(
    len(edges), len(trafficSegments), len(unqTrafficSegmentIDs))

Got 153 edges, 117 of which have traffic segments, of which 60 are unique.


### 3. Create Fake GPS traces

#### Extract coords from routes.shape, perturb, and save to dict

In [80]:
jsonDict = synthesize_gps(edges, shape, stddev=0)   # stddev in km

In [81]:
print 'Sending {0} fake GPS measurements to the reporter'.format(len(jsonDict['trace']))

Sending 154 fake GPS measurements to the reporter


### 4. Send Fake GPS to Reporter

In [82]:
baseUrl = 'http://reporter:8003/report'
report = requests.post(baseUrl, json=jsonDict)

In [83]:
segments = report.json()['segments']
print 'Reporter matched {0} GPS measurements to {1} OSMLR segments. Showing the first below: \n{2}'.format(
    len(jsonDict['trace']),len(segments), segments[3])

Reporter matched 154 GPS measurements to 14 OSMLR segments. Showing the first below: 
{u'segment_id': 200991413897, u'begin_shape_index': 17, u'start_time': 1495655781.0, u'length': 356, u'end_time': 1495655807.0, u'end_shape_index': 21}


### 5. Extract Segments and Speeds from Reporter Output

In [86]:
for segment in segments:
    if segment['length'] > 0 and segment['end_time'] > 0:
        segment['speed'] = segment['length'] / (segment['end_time'] - segment['start_time'])

In [87]:
[{segment['segment_id']:segment['speed']} for segment in segments if 'speed' in segment.keys()]

[{205655479945: 13.78125},
 {153310566025: 13.772727272727273},
 {200991413897: 13.692307692307692},
 {84288755616: 8.76023223299056},
 {70900537248: 4.180909375922962},
 {87677753248: 9.333333333333334},
 {190287550089: 14.846153846153847},
 {209749120649: 16.042780692490936},
 {226190792329: 3.297868169474078},
 {146901325728: 210.0},
 {147303978912: 210.0},
 {78383175584: 7.684210526315789}]